In [ ]:
#Import Python Libraries
#------------------------------
import numpy as np
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt

In [ ]:
#Read user_itemset_training.csv and Analyze
#------------------------------

data_u_is_train = pd.read_csv('./dataset/user_itemset_training.csv', sep=',', header=None)
data_u_is_train[0] = 'U'+data_u_is_train[0].astype(str)
data_u_is_train[1] = 'IS'+data_u_is_train[1].astype(str)
print(data_u_is_train.shape)
data_u_is_train.head()

In [ ]:
# checking jaccard_similarity -1
def jaccard_similarity(G, u, v):
    unbrs = set(G[u])
    vnbrs = set(G[v])
    return float(len(unbrs & vnbrs)) / len(unbrs | vnbrs)

In [ ]:
# checking jaccard_similarity -1
def jaccard_similarity1(s1, s2):
    s1 = set(s1)
    s2 = set(s2)
    return float(len(s1.intersection(s2))/len(s1.union(s2)))

In [ ]:
#user projected graph
data_u_is_train_set = {(x[0], x[1]) for x in zip(data_u_is_train[0], data_u_is_train[1])}

import networkx as nx
from networkx.algorithms import bipartite
X = data_u_is_train[0].unique()
Y = data_u_is_train[1].unique()


B = nx.Graph()
B.add_nodes_from(X, bipartite=0)
B.add_nodes_from(Y, bipartite=1)
B.add_edges_from(data_u_is_train_set)

G1 = bipartite.projected_graph(B, X)

In [ ]:
temp = list(G1.edges())

In [ ]:
train_proj = pd.DataFrame(temp, columns=['U1', 'U2'])

print('Train Dataframe')
train_proj.head()

In [ ]:
reversed_data = train_proj[['U2', 'U1']].rename(columns={'U2': 'U1', 'U1': 'U2'})
train_proj_total = pd.concat([train_proj, reversed_data], ignore_index=True)
train_proj_total.head()

In [ ]:
train_proj_total['NewCol'] = 1 / train_proj_total.groupby('U1')['U1'].transform('count')

In [ ]:
train_proj_total.head()

In [ ]:
data_u_is_valid_query = pd.read_csv('./dataset/user_itemset_valid_query.csv', sep=',', header=None)
data_u_is_valid_answer = pd.read_csv('./dataset/user_itemset_valid_answer.csv', sep=',', header=None)

data_u_is_valid_query[0] = 'U'+data_u_is_valid_query[0].astype(str)
data_u_is_valid_query[1] = 'IS'+data_u_is_valid_query[1].astype(str)

In [ ]:
data_u_is_valid_answer = data_u_is_valid_answer.rename({0: 'X'}, axis=1)
df_concat = pd.concat([data_u_is_valid_query, data_u_is_valid_answer], axis=1)

data_u_is_valid_df = df_concat.copy()
print('Valid Dataframe')
data_u_is_valid_df.head()

In [ ]:
import time
import csv
import operator
from itertools import islice

In [ ]:
start_time = time.time()

x_list = data_u_is_valid_df[0].tolist()
y_list = data_u_is_valid_df[1].tolist()
z_list = data_u_is_valid_df['X'].tolist()
save_ans = []

len_x_list = len(x_list)

print('trial', '%-7s'%'i', '%-7s'%'len', '%-7s'%'real', '%-7s'%'pred')

with open('user_itemset_valid_prediction.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    for i in range(len_x_list):
        print("----")
        print('i',i)
        x = x_list[i]
        y = y_list[i]
        z = z_list[i]
        ans1 = 0
        deno1 = 0

        rank1_list = list(set(train_proj_total[train_proj_total['U1'] == x]['U2']))
        for j in rank1_list:
            temp = list(train_proj_total[(train_proj_total['U1']==x)&(train_proj_total['U2']==j)]['NewCol'])[0]
            deno1+=temp
            ans1+=temp* (0 if data_u_is_train[(data_u_is_train[0] == j) & (data_u_is_train[1] == y)].empty else 1)
   
        writer.writerows([[ans1]])

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
data_u_is_test_query = pd.read_csv('./dataset/user_itemset_test_query.csv', sep=',', header=None)

data_u_is_test_query[0] = 'U'+data_u_is_test_query[0].astype(str)
data_u_is_test_query[1] = 'IS'+data_u_is_test_query[1].astype(str)

In [ ]:
start_time = time.time()

x_list = data_u_is_test_query[0].tolist()
y_list = data_u_is_test_query[1].tolist()
save_ans = []

len_x_list = len(x_list)

print('trial', '%-7s'%'i', '%-7s'%'len', '%-7s'%'real', '%-7s'%'pred')

with open('user_itemset_test_prediction.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    for i in range(len_x_list):
        print("----")
        print('i',i)
        x = x_list[i]
        y = y_list[i]
        ans1 = 0
        deno1 = 0

        rank1_list = list(set(train_proj_total[train_proj_total['U1'] == x]['U2']))
        for j in rank1_list:
            temp = list(train_proj_total[(train_proj_total['U1']==x)&(train_proj_total['U2']==j)]['NewCol'])[0]
            deno1+=temp
            ans1+=temp* (0 if data_u_is_train[(data_u_is_train[0] == j) & (data_u_is_train[1] == y)].empty else 1)
        
        if ans1>0.0005 : ans1 = 1
        else : ans1 = 0
        writer.writerows([[ans1]])

print("--- %s seconds ---" % (time.time() - start_time))
